# Detecting Potential Adverse Drug Reactions Using a Deep Neural Network Model

(Note -- this paper is different than the initial 3 project selection submitted since data was inaccessible across all 3 paper)

## Intro

On record, there are more than 2 million serious ADRs (adverse drug reactions) that occur amongst hospitalized patients which led to more than 100k deaths each year. Unfortunately since >90% of these ADRs go unreported, there lacks sufficient data for healthcare profession to property identify or predict potential ADRs.  This paper attempts to and focuses on using DNN to automatically detect potential ADR (adverse drug reactions) of a particular drug given its biological structures and other relevant infos with 2 goals: One is to idenfiy the potential ADRs of a drug given the history data adn context, and the other being predicting the possible ADRs of a new drug


This paper used a diverse sources of data to help identify and predict ADR. This includes clinical trials, electronic meidcal records, biomedical literature, chemical/molecular pathways of drugs.. all with handcrafted and careful feature selection. The paper also experimented with a number of classical ML models alongside DNN models to better compare and evaluate their relative performance


## Scope of Reproducibility

The scope of this first project draft is to build a MVP of the model using a subset of everything. This includes a subset of the data -- which means, instead of using both 2009 and 2012 data from SIDER as the training set, we will be using a subset of the 2009 data to start with. In addiion to that, we will also be using a subset of the input features to first test out the model and reduce the complexity-- this means that instead of using all of the features listed in the paper including biological, chemical, literature... findings of the drug, we will be focusing on using the chemical properties for the time being. (Note: This will indeed influence the accuracy and precision of the model training, but the whole goal is to make sure the base code run as is). Lastly, we will also be focusing on building one single model instead of reproing all model listed in the paper -- this means we will be focusing on using a basic RNN model instead of building out SVM, naives bayse.. etc etc

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

## Dependencies and pretrained model:

Not applicable

## Methodology: Data Description & Pre-processing:

The data required for this paper spawn across multiple places. For the scope of this project, we'd went to the following 3 places to retrieve the data (please see reference section below for actual instructions around data access):

1) SIDER -- from SIDER, we first downloaded a copy of the 2009 drug record and their relevant ADRs. This represents the Y record; and 

2) PubChem -- from PubChem, we downaloaded a copy of the 'chemical & physical properties' of the drugs. This is part of the X record

3) DrugBank -- from DrugBank, we requested for student research access for drug's bio properties. This consistutes part of the X feature set
(^ However it's unfortunate that the data under the xml format was unconvertable to CSV for further processing, so having access to the data in XML format was rather pointless)

(Note that the dimension of both X and Y in this project draft do not align with the original paper primarily because we'd reduced the scope of the data and the model)

Below few cells are used to load, clean, and transform the data. As we could see, since the data resides from different databases. Plenty effort was spent in prepping the data

In [2]:
#downloaded 2019 costart ADR data from SIDER and load it, transformed some of the data
df_y = pd.read_csv('costart_adverse_effects.tsv', sep = '\t')
df_y['pubchem'] = df_y['pubchem']* (-1)
df_y = df_y[['pubchem','drugname','ADR']] #filter it down to the drugname and ADR values

#downloaded 2019 chem properties from pubchem and load it, select the columns of data of interest as per the paper
df_x = pd.read_csv('PubChem_compound_properties.csv')
df_x = df_x[['cid','mw','xlogp', 'hbonddonor', 'hbondacc', 'rotbonds', 'polararea', 'complexity']]

/Users/lomischen/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (16,19,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Attempting to find the intersect of the drugs from X and Y, filtering out records that don't have record with ADRs 
# or record of chemical & physical properties

#Most of the following code are for pre processing and data cleaning

intersect_keys = sorted(list(set(df_y['pubchem']) & set(df_x['cid'])))
df_y = df_y[df_y['pubchem'].isin(intersect_keys)].sort_values(by = ['pubchem'])
df_x = df_x[df_x['cid'].isin(intersect_keys)]

In [4]:
total_num_cond = len(df_y['ADR'].unique())
total_num_drug = len(df_y['drugname'].unique())
Y = np.zeros((total_num_drug,total_num_cond))
Y2 = np.zeros((total_num_drug,1))

The followng cell focuses on creating a matrix/tensor of Y. It's one cell that takes up quite a bit of time. The hypothesis here is that the loop is rather computational expensive

In [5]:
dict_y = df_y.groupby('drugname').apply(lambda x:x.drop('drugname',axis=1).to_dict('list')).to_dict()
ordered_cond = df_y['ADR'].sort_values().unique() #ordering the list of ADR conditions, different than sorted drug names
unique_cid = df_x['cid']

drug_counter = 0 #counting the unique drug from CID numerical order
entry = 0 #counting the individual entries in the df_Y

for i in df_y['drugname']:
    if i == df_y['drugname'].unique()[drug_counter]:
        index = np.where(ordered_cond == df_y.iloc[entry][2])[0][0]
        #print(entry, drug_counter, i,df_y.iloc[entry][2], index)
        Y[drug_counter][index] = 1
        Y2[drug_counter] +=1
        entry +=1
    else:
        drug_counter +=1


In [6]:
df_x = df_x.fillna(0)
df_x = df_x.values.tolist()
Y2 = Y2.tolist()
Y2 = [item for sublist in Y2 for item in sublist]

## Implementation: Emulating the models built for in class projects

The following few cells is an attempt to create a model using the ones we'd built in class. This is also the base model used for the research paper with slight modification/variable. Ideally there would be multiple models so that we could compare the results against one another

We're also doing some modification on the base model. Instead of keeping the dimension of Y in its original form as is, I'd summed up the results and converted into a singular vector. This means that the embedded X are no longer mapping to binary results of Y but rather in the form of regression 

In [7]:
from torch.utils.data import Dataset
class CustomDataset(Dataset): 
    def __init__(self, seqs, hfs):
        self.x = seqs
        self.y = hfs   
    def __len__(self):
        return len(self.y)    
    def __getitem__(self, index):
        return (self.x[index],self.y[index])
        

dataset = CustomDataset(df_x, Y2)

In [8]:
def collate_fn(data):
    sequences, labels = zip(*data)
    y = torch.tensor(labels, dtype=torch.float)
    x = torch.tensor(sequences, dtype=torch.long)
    return x, y

In [9]:
#load data
from torch.utils.data import DataLoader
loader = DataLoader(dataset, batch_size=10, collate_fn = collate_fn)
loader_iter = iter(loader)
df_x, Y = next(loader_iter)

/var/folders/ln/tfnp7jpd7fqfkj79wjz21nb00000gn/T/ipykernel_30896/3764957986.py:4: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  x = torch.tensor(sequences, dtype=torch.long)


In [10]:
print('Shape of a batch x:', df_x.shape)
print('Shape of a batch y:', Y.shape)

Shape of a batch x: torch.Size([10, 8])
Shape of a batch y: torch.Size([10])


In [11]:
from torch.utils.data.dataset import random_split
split = int(len(dataset)*0.8)
lengths = [split, len(dataset) - split]
train_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 469
Length of val dataset: 118


In [12]:
def load_data(train_dataset, val_dataset, collate_fn):
    import torchvision
    import torchvision.datasets as datasets
    import torchvision.transforms as transforms   
    batch_size = 10    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn = collate_fn)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,collate_fn = collate_fn)    
    return train_loader, val_loader

train_loader, val_loader = load_data(train_dataset, val_dataset, collate_fn)

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 4) #None input size equals to the feature size of X
        self.fc2 = nn.Linear(4, 2) #None
        self.dropout = nn.Dropout(0.5) #None
        self.fc3 = nn.Linear(2,1) #None

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

# initialize the NN
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=8, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=2, out_features=1, bias=True)
)


In [16]:
model = Net()
criterion = nn.MSELoss()
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=0.012, momentum=0.9)


In [17]:
from sklearn.metrics import *

#input: Y_score,Y_pred,Y_true
#output: accuracy, auc, precision, recall, f1-score
def classification_metrics(Y_score, Y_pred, Y_true):
    acc, auc, precision, recall, f1score = accuracy_score(Y_true, Y_pred), \
                                           roc_auc_score(Y_true, Y_score), \
                                           precision_score(Y_true, Y_pred), \
                                           recall_score(Y_true, Y_pred), \
                                           f1_score(Y_true, Y_pred)
    return acc, auc, precision, recall, f1score



#input: model, loader
def evaluate(model, loader):
    model.eval()
    all_y_true = torch.LongTensor()
    #all_y_pred = torch.LongTensor()
    all_y_score = torch.FloatTensor()
    for x, y in loader:
        y_hat = model(x.float())
        # convert shape from [batch size, 1] to [batch size]
        #y_hat = y_hat.view(y_hat.shape[0])
        """
        TODO: obtain the predicted class (0, 1) by comparing y_hat against 0.5,
        assign the predicted class to y_pred.
        """

        #y_pred = torch.zeros(len(y_hat))
        y = y

        #for i in range(len(y_hat)):
         #   if y_hat[i] >= 0.5:
          #      y_pred[i] = 1
           # else:
            #    y_pred[i] = 0
            
        all_y_true = torch.cat((all_y_true, y.to('cpu').long()), dim=0)
        #all_y_pred = torch.cat((all_y_pred,  y_pred.to('cpu').long()), dim=0)
        all_y_score = torch.cat((all_y_score,  y_hat.to('cpu')), dim=0)
        
    MSE = mean_squared_error(all_y_true.detach().numpy(), all_y_score.detach().numpy() )
    #acc, auc, precision, recall, f1 = classification_metrics(all_y_score.detach().numpy(), 
     #                                                        all_y_pred.detach().numpy(), 
      #                                                       all_y_true.detach().numpy())
    #print(f"acc: {acc:.3f}, auc: {auc:.3f}, precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}")
    #return acc, auc, precision, recall, f1
    return MSE

In [18]:
# number of epochs to train the model
# feel free to change this (just make sure that Coursera does not timeout)
n_epochs = 5

# prep model for training
model.train()

train_loss_arr = []
for epoch in range(n_epochs):
    
    train_loss = 0
    for x, y in train_loader:
        """ Step 1. clear gradients """
        optimizer.zero_grad()
        
        """ 
        TODO: Step 2. perform forward pass using `model`, save the output to y_hat;
              Step 3. calculate the loss using `criterion`, save the output to loss.
        """
        y_hat = model.forward(x.float()) #None
        #print(y_hat)
        #y = y.unsqueeze(1)
        
        loss = criterion(y_hat,y)#None
        print(loss)
        
        """ Step 4. backward pass """
        loss.backward()
        """ Step 5. optimization """
        optimizer.step()
        """ Step 6. record loss """
        train_loss += loss.item()
        
    train_loss = train_loss / len(train_loader)
    if epoch % 20 == 0:
        train_loss_arr.append(np.mean(train_loss))
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
        evaluate(model, val_loader)

/var/folders/ln/tfnp7jpd7fqfkj79wjz21nb00000gn/T/ipykernel_30896/3764957986.py:4: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  x = torch.tensor(sequences, dtype=torch.long)
/Users/lomischen/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/lomischen/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, targe

tensor(15266.2559, grad_fn=<MseLossBackward0>)
tensor(8066.5586, grad_fn=<MseLossBackward0>)
tensor(14826.9990, grad_fn=<MseLossBackward0>)
tensor(8614.6895, grad_fn=<MseLossBackward0>)
tensor(7741.5142, grad_fn=<MseLossBackward0>)
tensor(6667.5264, grad_fn=<MseLossBackward0>)
tensor(18705.3398, grad_fn=<MseLossBackward0>)
tensor(4213.3794, grad_fn=<MseLossBackward0>)
tensor(3054.1978, grad_fn=<MseLossBackward0>)
tensor(3148.7744, grad_fn=<MseLossBackward0>)
tensor(3989.9846, grad_fn=<MseLossBackward0>)
tensor(5911.6919, grad_fn=<MseLossBackward0>)
tensor(4264.0103, grad_fn=<MseLossBackward0>)
tensor(7449.2261, grad_fn=<MseLossBackward0>)
tensor(7007.2046, grad_fn=<MseLossBackward0>)
tensor(2280.5122, grad_fn=<MseLossBackward0>)
tensor(4740.8511, grad_fn=<MseLossBackward0>)
tensor(2229.7368, grad_fn=<MseLossBackward0>)
tensor(4305.8989, grad_fn=<MseLossBackward0>)
tensor(10203.0723, grad_fn=<MseLossBackward0>)
tensor(7870.3052, grad_fn=<MseLossBackward0>)
tensor(3738.7988, grad_fn=<Mse

tensor(8044.7002, grad_fn=<MseLossBackward0>)
tensor(6070.2002, grad_fn=<MseLossBackward0>)
tensor(3784.2000, grad_fn=<MseLossBackward0>)
tensor(14248.2002, grad_fn=<MseLossBackward0>)
tensor(3848.5000, grad_fn=<MseLossBackward0>)
tensor(7321.2002, grad_fn=<MseLossBackward0>)
tensor(8205.5000, grad_fn=<MseLossBackward0>)
tensor(8662.5996, grad_fn=<MseLossBackward0>)
tensor(5324.3999, grad_fn=<MseLossBackward0>)
tensor(10826.7998, grad_fn=<MseLossBackward0>)
tensor(8611.7002, grad_fn=<MseLossBackward0>)
tensor(5314.2002, grad_fn=<MseLossBackward0>)
tensor(10284.2002, grad_fn=<MseLossBackward0>)
tensor(4912.2002, grad_fn=<MseLossBackward0>)
tensor(3110.7000, grad_fn=<MseLossBackward0>)
tensor(16676.8008, grad_fn=<MseLossBackward0>)
tensor(11357., grad_fn=<MseLossBackward0>)
tensor(5223.2002, grad_fn=<MseLossBackward0>)
tensor(4915., grad_fn=<MseLossBackward0>)
tensor(19362.4004, grad_fn=<MseLossBackward0>)
tensor(3050.8000, grad_fn=<MseLossBackward0>)
tensor(2419.3000, grad_fn=<MseLossBa

/Users/lomischen/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


## Computational Requirements:

Since the model created does not encapsulate all of the input features called for in the original paper, training and running the model was actually rather computational feasible. The paper was done using a regular mac air 13" 8 core CPU and no local/cloud GPU was required. The total runtime for training and evaluating the data took about ~ 5mins


## Result discussion
Unfortunately plenty of the time was spent on searching, prepping, and transforming data and mot much time was left for the actual model training and I'm still working towards debugging the code. So as of now there isn't sufficient result to share with the group. 

However once we finish debug the model, the next steps is to expand the dataset, supplement inputs with other feature selection, and train a few more models as per the paper ( e.g SVM and other DNNs with multiple layers) and compare their relevant result

## Discussion

Given the limitation of the data access as well as the ambiguity of data sources, the original paper was not successfully reproduced. To elaborate further, the struggle with data access was under 2 folds. For one, the 2nd dimension of the input feature X came under the XML format which was unconvertable to CSV for ease of analyitics despite spending extensive effort in doing so; second, the 3rd dimention of input feature X that involves using biomedical literature as inputs that was supposedly derived and scraped from thousands and millions of literatures without concrete references and pointers on how to do so made it impossible to repro. This means reproducing the result was deem to be a struggle. This also means that while a base model in attempt to repro the model was created, the results should be closely examined

What was easy
-- The research plan/goal as well as the overall experiments were relatively easy to comprehend. This means that I did not spend too much time and effort in attempt to understand what's being built and why/what except with the hows. Also we fortunately have in class experience building the base model that the paper used. So overall despite the fact that I was not able to repro the paper properly, it's still a paper that's relatively easy to comprehend where techniques learn from this class and previous classes were applicable. And with that base comprehension, I'd expect other scholars and ML researchers to be able to learn from it and extend on what's been done


What was difficult
-- The difficult portion boils down to multiple folds. Accessing data is at the top -- the features of X are scattered across multiple database and they require significant effort in cleaning and preprocessing. Not only that, even with accessiable data, not all of the features used and referenced in the paper were readily available. On top of that, there were no clear instructions and indication on what biomedical literatures were used as the 3rd input dimension of X especially when that was deemed to be the most contextual and helpful in training the data. 


Recommnendations to the orignnal authors for improving reproducibility
-- Suppose that enabling readers and other scholars to reproduce this paper is part of the authors' goals, I'd like to propose the authors to share or to provide clearer indication and instruction on how/where to access and clean the data so that others who are looking to repro the research could align with what's being used in the research paper. In addition to that, I'd also noticed that throughout the lab, the computational requiremenets for data pre-processing and training was slightly more demanding than what I'd anticipated. If the authors were able to provide some form of baseline or guidedance on CPU/GPU requirements, that would have made repro-ing the study a lot easier too.



## Data download instructions

1) Y -- retrieved from SIDER: http://sideeffects.embl.de/download/. One could read more about the download files via drug_names.tsv and drug_atc.tsv. The zipped file of meddra_all_indications.tsv.gz contains drug name and their relevant ADRs. Joining is required to join Y data across multiple years.

2) 1st dimension of input feature X -- retrieved from PubChem: https://pubchem.ncbi.nlm.nih.gov/classification/#hid=72. One could click into the section of Chemical and Physical Properties and download the data of interest. The downloaded data contains about 3/4 of the inputs required for the actual research. Cleaning and pre-processing is required 

3) 2nd dimension of input feature X: retrieved from DrugBank: https://go.drugbank.com/releases/latest. One would need to request special academic access before XML data could be downloaded. 


4) 3rd dimension of input feature X: (unknown)

## References

There's no code repo shown in the original research, so there was nothing for us to build off from and everything was completed from scratch

Original paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6381404/ ; 
PubChem database for chemical & phyiscal properties: https://pubchem.ncbi.nlm.nih.gov/classification/#hid=72 ;
DrugBank for bio properites: https://go.drugbank.com/releases/latest

